In [ ]:
import requests
import pandas as pd
import time
import json
import numpy as np

api_key = ''
request_header = {
    "User-Agent": "",
    "Accept-Language": "",
    "Accept-Charset": "",
    "Origin": "",
    "X-Riot-Token": ""
}


# api 호출시 사용되는 url
challenger = f"https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}"
r = requests.get(challenger)
league_df = pd.DataFrame(r.json())

league_df.reset_index(inplace=True)
league_entries_df = pd.DataFrame(dict(league_df['entries'])).T
league_df = pd.concat([league_df, league_entries_df], axis=1)

league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
league_df.info()
league_df.to_csv('그마데이터.csv',index=False,encoding = 'cp949')#중간저장

puuids = []

encrypted_summoner_id = league_df['summonerId']  

def summoner_v4_summoner_name(id):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{id}"
    response = requests.get(url, headers=request_header)

    if response.status_code == 200:
        summoner_data = response.json()
        return summoner_data['puuid']
    elif response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 5))
        print(f'Rate limit exceeded. Retrying after {retry_after} seconds.')
        time.sleep(retry_after)
        return summoner_v4_summoner_name(id)
    else:
        print(f'Error retrieving data for summoner ID: {id}', response.status_code)
        return None

for summoner_id in encrypted_summoner_id:
    puuid = summoner_v4_summoner_name(summoner_id)
    if puuid is not None:
        puuids.append(puuid)

test = pd.DataFrame(puuids)

test.to_csv('puuid.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          700 non-null    object
 1   summonerId    700 non-null    object
 2   summonerName  700 non-null    object
 3   leaguePoints  700 non-null    object
 4   wins          700 non-null    object
 5   losses        700 non-null    object
 6   veteran       700 non-null    object
 7   inactive      700 non-null    object
 8   freshBlood    700 non-null    object
 9   hotStreak     700 non-null    object
dtypes: object(10)
memory usage: 54.8+ KB
Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 43 seconds.
Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 44 seconds.


In [ ]:
puuids

['3Zn02Othql0GuDkdIY_nE7ldd_09Hwz5Zl5bBi_rUamm5kzAT8VgKnYU1V3Rk0pdLI_eID2_SlJiGA',
 '1J1hAuTfjdjJwIsAxCmJxTRzPFsAQBveRIQI3RLVWpo2xViSPW-QIfYnyfXzX3tDEfsnDFjITRmAPQ',
 'JYaK2j6L9ZWJdApFn2jcaAu9ykdMrIqXoL5_o_VgmlDsLOxybkbk-aySME8tqhoZt8gpnScQO21WfQ',
 '7BeXCIz-j6x37qxGdgG2wjx0wqyRUMeiTXzIh7POH9D2KqulqCUaFL-Nuqm8KM2o38H0RG-sLIN7ZA',
 'bseCoI6AstOzr8-eblPdyi6du9Vd2kjfnWFb5WWmyRT7eIrRxj_0D-0rxaj0H1TIPc3qTLG53W3NXA',
 'HzxygPsGh_YC9UWtlLy0pGaHYF_SBJlrHBirlky8cVqB0bnKTjFD5SdHYDVA3sUlwg2LbXRLDM4CzQ',
 'mQ5UxWVkFoHKPBRW2Si1UK2VBolSUYTUvP5K3DCTdu6fWTw8El61DjFUXTRCKGZ__0IRfqV7FDkjIw',
 'E3Q2vk90_bg5-I7XIcrhsm8hqZoiqCCf1JMn8KWJOyYmw7_Csjb_nDioCylwMLNLiqFRT58KUUVq3w',
 'iKPaS-kyB7H1QHeOrMAD3EfEk2kcT9s-YiM0l_Dfmlh7ihX8I9rI-nsO6lsrcjwcvXIg2UbMQ5btZg',
 'WtJU5hiifvHd695_SAdkWWu1G9-gIkqFKeiWutTrzDnTzU4GHcK4yWAT_yRo_GunbHo6TnZLuHRvvQ',
 'N5B8ZADnHzEaOjzRh7tD9hHhbmYdIawNW2SN5pBShcJ3_A5zC-Y4OK-igE7GZyC7Ik9aXypsJ7WWjA',
 'QnixxQb3ddaQUnKAiAQPJUGigyl8t3-1nQNte8jdYUGvSa6RG07ULkS5rCLo0gZdYZ9OVACulLcTpg',
 'wo

In [ ]:
#계정정보(PUUID)로 게임 MATCH ID 넘겨받기

match_second = []


def match_v5_get_list_match_id(puuid, start, count):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    response = requests.get(url, headers=request_header)
    
    if response.status_code == 200:
        match_ids = response.json()
        return match_ids
    elif response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 5))
        print(f'Rate limit exceeded. Retrying after {retry_after} seconds.')
        time.sleep(retry_after)
        return match_v5_get_list_match_id(puuid, start, count)
    else:
        print(f'Error retrieving match IDs for PUUID: {puuid}. Status code: {response.status_code}')
        return None

for puuid in puuids:
    match_ids = match_v5_get_list_match_id(puuid,0,30)
    if match_ids is not None:
        match_second.append(match_ids)



Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 41 seconds.
Rate limit exceeded. Retrying after 44 seconds.
Rate limit exceeded. Retrying after 43 seconds.
Rate limit exceeded. Retrying after 43 seconds.
Rate limit exceeded. Retrying after 43 seconds.


In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
match_second_test = pd.DataFrame(match_second)
match_second_test.to_csv('기록.csv')


ma = match_second_test.iloc[:,0]

ma


0      KR_6501204701
1      KR_6501187171
2      KR_6501060821
3      KR_6500569631
4      KR_6499523151
           ...      
695    KR_6491150474
696    KR_6501178600
697    KR_6501163313
698    KR_6501079526
699    KR_6501245530
Name: 0, Length: 700, dtype: object

In [ ]:
#해당 게임(Match ID) 에 관련된 모든 정보 받기
match_history = []

def match_v5_get_match_history(matchId):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchId}"
    response = requests.get(url, headers=request_header)

    if response.status_code == 200:
        match_data = response.json()
        return match_data
    elif response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 5))
        print(f'Rate limit exceeded. Retrying after {retry_after} seconds.')
        time.sleep(retry_after)
        return match_v5_get_match_history(matchId)
    else:
        print(f'Error retrieving data for match ID: {matchId}. Status code: {response.status_code}')
        return None

for match_ids in ma:
    match_data = match_v5_get_match_history(match_ids)
    if match_data is not None:
        match_history.append(match_data)

print(match_history)


Rate limit exceeded. Retrying after 43 seconds.
Rate limit exceeded. Retrying after 39 seconds.
Rate limit exceeded. Retrying after 39 seconds.
Rate limit exceeded. Retrying after 39 seconds.
Rate limit exceeded. Retrying after 39 seconds.
Rate limit exceeded. Retrying after 40 seconds.
Rate limit exceeded. Retrying after 38 seconds.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
match_history

In [ ]:
data_history = pd.DataFrame(match_history)

data_history = data_history['info']
print(type(data_history))

data = []
for row in data_history: 
  data.append(row.values())
    
df = pd.DataFrame(data)
df.columns = data_history[0].keys()  # 처음 행에서 키만 추출해서 컬럼명으로 설정

In [ ]:
data_history

In [ ]:
df.head()

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,1684440732402,2035,1684442797829,6501204701,CLASSIC,teambuilder-match-6501204701,1684440762471,MATCHED_GAME,13.10.509.8402,11,"[{'allInPings': 0, 'assistMePings': 9, 'assist...",KR,420,"[{'bans': [{'championId': 902, 'pickTurn': 1},...",
1,1684436253921,2220,1684438504796,6501187171,CLASSIC,teambuilder-match-6501187171,1684436284731,MATCHED_GAME,13.10.509.8402,11,"[{'allInPings': 0, 'assistMePings': 0, 'assist...",KR,420,"[{'bans': [{'championId': 119, 'pickTurn': 1},...",
2,1684427204590,1776,1684429017423,6501060821,CLASSIC,teambuilder-match-6501060821,1684427241345,MATCHED_GAME,13.10.509.8402,11,"[{'allInPings': 0, 'assistMePings': 3, 'assist...",KR,430,"[{'bans': [], 'objectives': {'baron': {'first'...",
3,1684410423915,1760,1684412221152,6500569631,CLASSIC,teambuilder-match-6500569631,1684410460193,MATCHED_GAME,13.10.509.8402,11,"[{'allInPings': 0, 'assistMePings': 4, 'assist...",KR,420,"[{'bans': [{'championId': 518, 'pickTurn': 1},...",
4,1684332498868,2409,1684334938413,6499523151,CLASSIC,teambuilder-match-6499523151,1684332528853,MATCHED_GAME,13.9.506.4846,11,"[{'allInPings': 1, 'assistMePings': 12, 'assis...",KR,420,"[{'bans': [{'championId': 81, 'pickTurn': 1}, ...",


In [ ]:
type(df)

In [ ]:
b=[]
for i in range(1,200):
  a=pd.DataFrame(df['participants'][i])
  b.append(a)
b = pd.DataFrame(b)

In [ ]:
# c = pd.DataFrame(b)
print(type(b))

b

<class 'list'>


[   allInPings  assistMePings  assists  baitPings  baronKills  basicPings  \
 0           0              0        2          0           0           0   
 1           0              7       10          0           1           0   
 2           1             17        6          0           0           0   
 3           0              0        5          0           0           0   
 4           0              3        8          0           0           0   
 5           0              4       14          0           0           0   
 6           0              7       10          0           1           0   
 7           0              3        8          0           0           0   
 8           0              2       14          0           0           0   
 9           2              6       22          0           0           0   
 
    bountyLevel                                         challenges  \
 0            0  {'12AssistStreakCount': 0, 'abilityUses': 717,...   
 1         

In [ ]:
# df_a = df['participants'][0]
df_a = pd.DataFrame(df['participants'][0])
df_a.head()   

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,9,8,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 381,...",17486,17,...,4,5,10,0,0,45,9,15,15,False
1,0,5,6,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 638,...",17677,17,...,1,1,10,0,0,36,2,13,4,False
2,0,0,9,0,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 582,...",13892,15,...,0,0,10,0,0,16,0,4,4,False
3,0,5,10,1,0,0,0,"{'12AssistStreakCount': 0, 'abilityUses': 228,...",17309,17,...,0,0,10,0,0,37,5,14,9,False
4,0,11,16,2,0,0,0,"{'12AssistStreakCount': 1, 'abilityUses': 116,...",10233,13,...,0,1,10,0,0,84,9,10,36,False


In [ ]:
for row in data_history: 
  data.append(row.values())
    
df = pd.DataFrame(data)
df.columns = data_history[0].keys()  # 처음 행에서 키만 추출해서 컬럼명으로 설정

In [ ]:
# df_a = df['participants'][0]
df_a = pd.DataFrame(df['participants'][0])
df_a.head()

In [ ]:
# print(df_a)
ex_b.to_csv('ex.csv')

In [ ]:
for item in a:
  for key, value in item.items():
    